# 修改shp文件中的英文内容

In [2]:
import geopandas as gpd
import pandas as pd
import requests
import hashlib
from tqdm import tqdm

In [105]:
data = gpd.read_file("/Users/creative/Downloads/gadm36_IND_shp/gadm36_IND_3.shp")

In [106]:
data.head(5)

,NAME_0,NAME_1,NAME_2,NAME_3,geometry
0,India,Andaman and Nicobar,Nicobar Islands,n.a. ( 2304),"MULTIPOLYGON (((92.78778 9.24417, 92.78889 9.2..."
1,India,Andaman and Nicobar,North and Middle Andaman,n.a. ( 2178),"MULTIPOLYGON (((93.64841 14.93487, 93.64917 14..."
2,India,Andaman and Nicobar,South Andaman,n.a. ( 2178),"MULTIPOLYGON (((93.83970 12.32082, 93.85775 12..."
3,India,Andhra Pradesh,Anantapur,Anantapur,"POLYGON ((77.35452 14.52155, 77.34958 14.52136..."
4,India,Andhra Pradesh,Anantapur,Dharmavaram,"POLYGON ((77.35341 14.27068, 77.35244 14.28055..."


## 百度翻译接口

http://api.fanyi.baidu.com/doc/21

In [15]:
def translation_baidu(query):
    import json
    url = "http://api.fanyi.baidu.com/api/trans/vip/translate?"
    appid = "20210813000915707"
    salt = "662"
    sign = "CVh7Y0xhL6LZUBzuvUZH"
    key = hashlib.md5(str(appid + query + salt + sign).encode(encoding='UTF-8')).hexdigest() #加密密钥
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36"}
    keyword = {"q": query, "from": "en", "to": "zh","appid":appid,"salt":salt,"sign":key}
    
    b = requests.get(url, headers=headers, params=keyword)
    if b.status_code == 200:
        html = b.text
        da = json.loads(html)["trans_result"][0]["dst"]
        return str(da)

## 翻译部分

判断字段类型，对于文本类型字段进行翻译

https://www.jianshu.com/p/8a5f0710cad3

获取唯一值字典，然后进行数据匹配

In [101]:
def dicts(shp,dict_add):
    ziduan = shp.dtypes[shp.dtypes == object].index.tolist()
    #生成翻译字典
    dicts = {}
    for i in tqdm(ziduan):
        for trans in shp[i].unique():
            try:
                dicts.update({trans:str(translation_baidu(trans))})
            except:
                pass 
    # 引入自定义字典
    try:
        AD = pd.read_csv(dict_add)
        AD = AD.set_index('key').to_dict()["value"]
    except:
        AD = {}
    # 删除翻译器的结果
    duplicate = list(dicts.keys() & AD.keys())
    for i in range(len(duplicate)):
        dicts.pop(duplicate[i])
    dicts.update(AD)
    
    return dicts 

In [102]:
def fix_shp(shp,dicts):
    ziduan = shp.dtypes[shp.dtypes == object].index.tolist()
    for i in tqdm(ziduan):
        for dxf in range(len(shp)):
            try:
                shp.loc[dxf,i] = dicts[shp.loc[dxf,i]]
            except:
                pass
    return shp

In [103]:
fix_shp(data,dicts(data,""))

100%|██████████| 4/4 [00:02<00:00,  1.48it/s]


,NAME_0,NAME_1,NAME_2,NAME_3,geometry
0,印度,安达曼和尼科巴群岛,尼科巴群岛,n、 a.（2304）,"MULTIPOLYGON (((92.78778 9.24417, 92.78889 9.2..."
1,印度,安达曼和尼科巴群岛,安达曼北部和中部,n、 a.（2178）,"MULTIPOLYGON (((93.64841 14.93487, 93.64917 14..."
2,印度,安达曼和尼科巴群岛,南安达曼,n、 a.（2178）,"MULTIPOLYGON (((93.83970 12.32082, 93.85775 12..."
3,印度,安得拉邦,阿南塔普尔,阿南塔普尔,"POLYGON ((77.35452 14.52155, 77.34958 14.52136..."
4,印度,安得拉邦,阿南塔普尔,达摩瓦兰,"POLYGON ((77.35341 14.27068, 77.35244 14.28055..."
...,...,...,...,...,...
2335,印度,西孟加拉邦,帕尔干纳南24号,迪亚蒙德哈尔博乌尔,"MULTIPOLYGON (((88.41639 21.89883, 88.41638 21..."
2336,印度,西孟加拉邦,帕尔干纳南24号,n、 a.（1187）,"MULTIPOLYGON (((88.56661 21.91009, 88.56715 21..."
2337,印度,西孟加拉邦,帕尔干纳南24号,n、 a.（1229）,"MULTIPOLYGON (((88.64000 21.92168, 88.64027 21..."
2338,印度,西孟加拉邦,北方迪纳杰普尔酒店,伊斯拉姆普尔,"POLYGON ((88.18713 26.02453, 88.18711 26.02406..."
